# Processing Application
## Display mean values of yearly raster data into user selected area.

This application makes use of the **ipyleaflet** and **rasterio** packages in addition to owslib. Folium is not used here. The user is prompted to select a bounding box inside the map. The coordinates are updated automatically. The application goes on downloading 

## WMS with geoserver initiation as normal

In [1]:
# WMS for NDVI layer from geoserver
from owslib.wms import WebMapService
url = 'http://localhost:8082/geoserver/ows'
wms = WebMapService(url, version='1.1.1')

I choose raster layers and styles here:

In [2]:
dni = "DNI_yearly"
pvout = "PVOUT_yearly"
temp = "TEMP_yearly"
style = 'geonode:3color_gradient_PVOUT'
print("The following map consists of three layers:\n")
[print("Layer",i,":\n"+wms["geonode:"+layer].abstract, "\n") for (layer,i) in zip([pvout, temp, dni], [1,2,3])]

The following map consists of three layers:

Layer 1 :
Longterm yearly average of potential photovoltaic production (PVOUT) in kWh/kWp, covering the period 1994-2018 

Layer 2 :
Longterm yearly average of air temperature (TEMP) in °C, covering the period
1994-2018 

Layer 3 :
Longterm yearly average of direct normal irradiation (DNI) in kWh/m2, covering
the period 1994-2018 



[None, None, None]

In [6]:
from ipyleaflet import Map, Polygon, WMSLayer, LayersControl, basemaps, FullScreenControl
from IPython.core.display import display, HTML

center = [38.2, 23.5]
zoom=6

countries = WMSLayer(
    url=url,
    layers="geonode:ne_10m_admin_0_countries",
    format='image/png',
    transparent=True,
    attribution= "Eurostat, Natural Earth, Countries Data 2016",
    opacity=0.1,
)

m = Map(center=center, zoom=zoom, basemap=basemaps.MtbMap, name='Main map')

pvout_l = WMSLayer(
    url=url,
    layers="geonode:"+pvout,
    format='image/png',
    transparent=True,
    opacity=0.1,
    style=style,
    control=True,
    name="PVOUT",
    show=False
)

temp_l = WMSLayer(
    url=url,
    #layers='geonode:NDVI_20100101',
    layers="geonode:"+temp,
    format='image/png',
    transparent=True,
    opacity=0.1,
    show=True,
    control=True,
    name="Temperature"
)

dni_l = WMSLayer(
    url=url,
    layers="geonode:"+dni,
    format='image/png',
    transparent=True,
    opacity=0.1,
    style=style,
    show=False,
    control=True,
    name="DNI"
)

m.add_layer(dni_l)
m.add_layer(pvout_l)
m.add_layer(temp_l)


control = LayersControl(position='topright')
m.add_control(control)
m.add_control(FullScreenControl())

I create a polygon that can be transformed (i.e. rotated and scaled) and dragged. You can drag the polygon around, or use the handler to rotate it and to scale it. Note that the transformations are synced accross all the map views.

In [7]:
polygon_coords = [
    [37, 23],
    [37, 24],
    [38, 24],
    [38, 23]]

pg = Polygon(locations=polygon_coords, transform=True, draggable=True, name="Selection Box")
m.add_layer(pg)

print("Please move polygon in order to select area for advanced aggregated statistics:")
m

Please move polygon in order to select area for advanced aggregated statistics:


Map(center=[38.2, 23.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out…

In this section I get the bbox coordinates as defined by the user:

In [5]:
if isinstance(pg.locations[0][0], dict):
    bbox = (pg.locations[0][0]['lng'], 
            pg.locations[0][0]['lat'],
            pg.locations[0][2]['lng'],
            pg.locations[0][2]['lat'])
else:
    bbox = (pg.locations[0][1],
            pg.locations[0][0],
            pg.locations[2][1],
            pg.locations[2][0])
print("The selected bounding box will be downloaded using WCS for the 3 layers "\
      "in the next section:\n\nBounding box coordinates: {}".format(bbox))

The selected bounding box will be downloaded using WCS for the 3 layers in the next section:

Bounding box coordinates: (23, 37, 24, 38)


I download the specific bounding box from the raster layer from geoserver using the download functions presented in exercise 2.

### WCS connection and download service for the selected area

In [6]:
from owslib.wcs import WebCoverageService
import webbrowser
import os

wcs = WebCoverageService(url=url, version='1.0.0')

def wcs_download(layer, bbox=None, workspace="geonode", format="GeoTIFF", height=1024, width=1024):
    if bbox:
        response = wcs.getCoverage(identifier="geonode"+":"+layer, format=format, height=height, width=width, crs="EPSG:4326", bbox=bbox)
    else:
        response = wcs.getCoverage(identifier="geonode"+":"+layer, format=format, height=height, width=width, crs="EPSG:4326")
    webbrowser.open(response.geturl())
    return response.geturl()

The user is prompted to the download page and has to store the downloaded files in a folder named with relative "/wcs".

In [7]:
wcs_download(pvout, bbox=bbox if bbox else None)
wcs_download(temp, bbox=bbox if bbox else None)
wcs_download(dni, bbox=bbox if bbox else None)

'http://localhost:8082/geoserver/wcs?version=1.0.0&request=GetCoverage&service=WCS&Coverage=geonode%3ADNI_yearly&BBox=23%2C37%2C24%2C38&crs=EPSG%3A4326&format=GeoTIFF&width=1024&height=1024'

## Extract mean values for each raster layer

In [8]:
import rasterio
import os
import numpy as np

temp_r = rasterio.open(os.path.join('wcs', 'geonode_' + temp + '.tif'))
dni_r = rasterio.open(os.path.join('wcs', 'geonode_' + dni + '.tif'))
pvout_r = rasterio.open(os.path.join('wcs', 'geonode_' + pvout + '.tif'))

Check Bounding Box

In [9]:
pvout_r.bounds

BoundingBox(left=22.105871016944615, bottom=37.84372860643055, right=22.484591017825565, top=38.09970041593117)

Parse raster data into numpy arrays

In [10]:
dni_band = dni_r.read(1)
temp_band = temp_r.read(1)
pvout_band = pvout_r.read(1)
np.shape(pvout_band)

(1024, 1024)

## Extract mean raster values over bbox area excluding empty areas nan values

### Mean Temperature

In [101]:
temp_mean = temp_band[~np.isnan(temp_band)]
print ("The mean temperature in the selected area is {:.3f} Celsius degrees".format(np.mean(temp_mean)))

The mean temperature in the selected area is 10.237 Celsius degrees


### Mean Potential Photovoltaic production

In [102]:
pvout_mean = pvout_band[~np.isnan(pvout_band)]
print ("The mean potential photovoltaic production in the selected area is {:.3f} kWh/kWp".format(np.mean(pvout_mean)))

The mean potential photovoltaic production in the selected area is 1491.729 kWh/kWp


### Mean Direct Normal Irradiation

In [103]:
dni_mean = dni_band[~np.isnan(dni_band)]
print ("The mean direct normal irradiation in the selected area is {:.3f} kwh/m^2".format(np.mean(dni_mean)))

The mean direct normal irradiation in the selected area is 1575.812 kwh/m^2
